In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math
from scipy.optimize import minimize
from scipy.spatial import ConvexHull
from scipy.linalg import norm
from scipy.optimize import minimize_scalar

data = pd.read_csv("banana.csv", sep=',')

In [8]:
def weak_learner(X, y, w):
    # Define and train the weak learner
    clf = DecisionTreeClassifier(max_depth=2)
    clf.fit(X, y,sample_weight=w)
    return clf

In [9]:
def encode_hypothesis(hypothesis, num_hypotheses, h):
    encoded_vector = np.zeros(num_hypotheses)
    encoded_vector[hypothesis] = h[hypothesis]
    return encoded_vector

In [10]:
def delta(d):
    m = len(d)
    return np.sum(d * np.log(d)) + np.log(m)

def expression(d):
    return np.dot(d.T, np.dot(A2, w)) + (1/n) * delta(d)

def minimize_expression(A2, w_1, n, v):
    m = A2.shape[0]  # Numero di righe nella matrice A
    P_m_v = np.random.rand(m) / v  # Generazione casuale di d in [0, 1/v]^m (ad esempio)
    best_d = None
    min_value = float('inf')
    for _ in range(m):  # Prova vari valori casuali di d
        d = P_m_v / np.sum(P_m_v)  # Normalizza per rispettare Σₖ dₖ¹ = 1
        value = expression(d)
        
        if value < min_value:
            min_value = value
            best_d = d

    return d, best_d

In [11]:
def secondary_weight(A, d, w):
    return np.dot(d.T, np.dot(A, w))

def minimize_secondary_weight(A, w):
    m = A.shape[0]
    best_d = None
    min_value = float('inf')

    for _ in range(m):
        value = secondary_weight(A2, d, w)
        if value < min_value:
            min_value = value
            best_d = d

    return best_d, min_value

def maximize_w_in_convex_hull(E_t_plus_1, A2, w):
    max_value = -float('inf')
    max_w = None
    for w_candidate in E_t_plus_1:
        d, value = minimize_secondary_weight(A2, w_candidate)
        if value > max_value:
            max_value = value
            max_w = w_candidate
    return max_w, max_value



In [12]:
def f(w):
    return np.dot(d.T, np.dot(A2, w))
def update_weights(f_w):
    best_w = None
    min_value = float('inf')
    s = A2.shape[1]
    for c in range(len(f_w)):
        for _ in range(s):
            value = f(f_w[c])
            if value < min_value:
                min_value = value
                best_w = f_w[c]
    return best_w, min_value

In [13]:
#Normalizing the variables to values in [-1,1]
for col in data.columns:
    if col != 'Class':
        min_val = data[col].min()
        max_val = data[col].max()
        data[col] = (data[col] - min_val) / (max_val - min_val)  # Normalize to range 0-1
        data[col] = data[col] * 2 - 1  # Rescale to range -1 to 1

In [14]:
#Data preparation 
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

X_test = test_data.drop('Class', axis=1)
y_test = test_data['Class']

In [15]:
# Creazione di un albero decisionale
tree = DecisionTreeClassifier(max_depth=2)
tree.fit(X_train, y_train)

# Ottieni la matrice di regole di decisione (A)
regole_matrice = tree.tree_.threshold.reshape(-1, 1)

# Stampa la matrice di regole di decisione
print("Matrice di regole di decisione:")
print(regole_matrice)

Matrice di regole di decisione:
[[ 0.07150538]
 [-0.44767025]
 [-2.        ]
 [-2.        ]
 [ 0.5440678 ]
 [-2.        ]
 [-2.        ]]


In [16]:
X_train_array = X_train.values

In [17]:
# Numero di righe nei dati di test
n_righe = X_train.shape[0]

# Numero di regole
n_regole = 7

# Matrice per memorizzare le previsioni per ogni riga e regola
previsioni_totali = np.zeros((n_righe, n_regole))

# Calcola le previsioni per ciascuna riga e ciascuna regola
for i in range(n_regole):
    previsioni_totali[:, i] = tree.predict(X_train_array * regole_matrice[i])

print("Previsioni totali:")
print(previsioni_totali)


Previsioni totali:
[[ 1.  1. -1. ... -1. -1. -1.]
 [ 1.  1. -1. ... -1. -1. -1.]
 [ 1. -1. -1. ... -1. -1. -1.]
 ...
 [ 1.  1.  1. ...  1.  1.  1.]
 [ 1.  1.  1. ...  1.  1.  1.]
 [ 1. -1. -1. ...  1. -1. -1.]]


C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\m

In [18]:
previsioni_totali = pd.DataFrame(previsioni_totali)
A2 = previsioni_totali
print(A2)

        0    1    2    3    4    5    6
0     1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0
1     1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0
2     1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
3     1.0 -1.0  1.0  1.0  1.0  1.0  1.0
4     1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0
...   ...  ...  ...  ...  ...  ...  ...
4235  1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0
4236  1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0
4237  1.0  1.0  1.0  1.0  1.0  1.0  1.0
4238  1.0  1.0  1.0  1.0  1.0  1.0  1.0
4239  1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0

[4240 rows x 7 columns]


In [19]:
#Train the weak learner : decision tree of max depth 2 
weak_classifier = weak_learner(X_train, y_train,1)
y_pred_weak = weak_classifier.predict(X_test)
print("Predictions:", y_pred_weak)

Predictions: [-1 -1 -1 ... -1  1 -1]


In [20]:
#set d0
m = X_train.shape[0]
n = len(regole_matrice)
d0 = np.zeros(m)
for i in range(m):
    d0[i] = 1/m 
#sendo d0 to the weak learner to obtain hj_1
h= weak_learner(X_train,y_train,d0)
hj_1 = h.tree_.threshold.reshape(-1, 1)


In [21]:
hj_1_index = np.argmax(hj_1)  # Find the index of the predicted hypothesis
num_hypotheses = len(hj_1) # Total number of hypotheses in the set H
ej1 = encode_hypothesis(hj_1_index, num_hypotheses, hj_1)

In [22]:
w = np.zeros(m,)
w=ej1

In [ ]:
v = 100
d, result = minimize_expression(A2, w, n, v)

In [ ]:
#obtain the hypotesis hj t+1 
h2= weak_learner(X_train,y_train,result)
hj_2 = h2.tree_.threshold.reshape(-1, 1)

In [ ]:
hj_2_index = np.argmax(hj_2)  # Find the index of the predicted hypothesis
num_hypotheses = len(hj_2)  # Total number of hypotheses in the set H

ej2 = encode_hypothesis(hj_2_index, num_hypotheses, hj_2)

In [ ]:
print(hj_1)
print(hj_2)
print(ej1)
print(ej2)

In [ ]:
#compute the FW weight
lamb= []
value_0 = np.dot(A2, (ej2-w))
print("value0 is: {}".format(value_0))
value_1 = np.dot(result.T, value_0)
print("value1 is: {}".format(value_1))
value_2 = np.dot(A2, (ej2 - w))
print("value2 is: {}".format(value_2))
value_3 = norm(value_2, ord=np.inf)
print("value3 is: {}".format(value_3))
value = value_1 / value_3
print("value is: {}".format(value))
lamb.insert(0, np.clip(value, 0, 1))
print("lamb is: {}".format(lamb))
w_1= []
r = (ej2 - w)
print("r is: {}".format(r))
w_1 = w + (lamb*r)
print("w_1 is: {}".format(w_1))

In [ ]:
E_t_plus_1 = [ej1, ej2]

In [ ]:
result_w, result_value = maximize_w_in_convex_hull(E_t_plus_1, A2, w)
print("Vettore w ottimale:", result_w)
print("Valore massimo:", result_value)

In [ ]:
w_2 = result_w
f_w = []
f_w.insert(0, w_1)
f_w.insert(1, w_2)
f_w = [array.tolist() for array in f_w]

In [ ]:
best_w, v = update_weights(f_w)

In [ ]:
best_w

In [ ]:
#set v, T parameters
v = 100
T = 100
#create all the variables that we need
E_t = []
E_t_plus_1 = [ej1]
lamb = []
lamb.insert(0, np.zeros(m))
for t in range(1, T):
    print("iteration number:{}".format(t))
    #create the distribution d_t
    d, result = minimize_expression(A2, w, n, v)
    #obtain the hypothesis h_j+1 and e_j+1
    h2= weak_learner(X_train,y_train,result)
    hj_2 = h2.tree_.threshold.reshape(-1, 1)  # Total number of hypotheses in the set H
    hj_2_index = np.argmax(hj_2)  # Find the index of the predicted hypothesis
    num_hypotheses = len(hj_2)  # Total number of hypotheses in the set H
    ej2 = encode_hypothesis(hj_2_index, num_hypotheses, hj_2)
    E_t_plus_1.insert(t, ej2)
    #compute the FW weight
    lamb= []
    value_0 = np.dot(A2, (ej2 - w))
    value_1 = np.dot(result.T, value_0)
    value_2 = np.dot(A2, (ej2 - w))
    value_3 = norm(value_2, ord=np.inf)
    value = value_1 / value_3
    lamb.insert(0, np.clip(value, 0, 1))
    w_1= []
    r = (ej2 - w)
    w_1 = w + (lamb*r)
    print("The FW weight is: {}".format(w_1))
    result_w, result_value = maximize_w_in_convex_hull(E_t_plus_1, A2, w)
    w_2 = result_w
    print("The secondary weight is{}".format(w_2))
    f_w = []
    f_w.insert(0, w_1)
    f_w.insert(1, w_2)
    f_w = [array.tolist() for array in f_w]
    best_w, v = update_weights(f_w)
    w = best_w

The FW weight is: [0.         0.         0.         0.         0.56858416 0.
 0.        ]
The secondary weight is[0.        0.        0.        0.        0.5440678 0.        0.       ]


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_10860\3156113329.py:25: RuntimeWarning: invalid value encountered in scalar divide
  value = value_1 / value_3


The FW weight is: [nan nan nan nan nan nan nan]
The secondary weight is[0.        0.        0.        0.        0.5440678 0.        0.       ]
The FW weight is: [0.         0.         0.         0.         0.55043063 0.
 0.        ]
The secondary weight is[0.         0.         0.         0.         0.55762714 0.
 0.        ]
The FW weight is: [0.         0.         0.         0.         0.55043063 0.
 0.        ]
The secondary weight is[0.         0.         0.         0.         0.55762714 0.
 0.        ]
The FW weight is: [0.         0.         0.         0.         0.55043063 0.
 0.        ]
The secondary weight is[0.         0.         0.         0.         0.55762714 0.
 0.        ]
The FW weight is: [0.         0.         0.         0.         0.55043063 0.
 0.        ]
The secondary weight is[0.         0.         0.         0.         0.55762714 0.
 0.        ]
The FW weight is: [0.         0.         0.         0.         0.55043063 0.
 0.        ]
The secondary weight is[0. 